# Guide to useful functions for reflectometry measurements of quantum dots

By Theodor Lundberg, February 2021
 
Note that the examples within do not run as-is because they rely on connection to an existing qcodes database and to instances of instruments in an experimental setup. The code must therefore be adjusted slightly such that is connects to your database and to your instruments.

# Standard imports

In [ ]:
# Import standard useful libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import time
from lmfit import Model

# Import qcodes
import qcodes as qc
from qcodes import Station, load_or_create_experiment, \
    initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at
from qcodes.instrument.base import find_or_create_instrument
from qcodes import load_last_experiment, load_experiment, new_experiment, experiments
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes.logger.logger import start_all_logging
import qcodes.instrument_drivers
import qcodes.utils.validators as qcValidator

# Import useful functions
from functions.doNd_wSubplot import do0d, do1d, do2d, subplot_by_id_2, subplot_by_id_3, subplot_by_id_4

The functions we introduce later, will require the qcodes experiment instance, and therefore we show how this can be defined:

In [ ]:
cwd = os.getcwd()
initialise_or_create_database_at(os.path.join(cwd, 'theosQCodesData.db'))
qc.config["core"]["db_location"] = os.path.join(cwd, 'theosQCodesData.db')
exp = load_or_create_experiment(experiment_name='20200731 - DQD', sample_name="3")

# Optimising reflectometry signal as a function of magnetic field

In dispersive readout with superconducting inductors, applying a magnetic field leads to increased kinetic inductance and hence smaller resonance frequencies. To retain the optimal readout signal, the probe frequency must therefore be updated as a function of magnetic field.

#### 0. Load a table where one can look up the reflectomtry resonance frequency of the resonator at a given magnetic field

IMPORTANT: 
The look-up array should have B field values in 1st column and corresponding approximate drive frequencies in 2nd column 

In [ ]:
cwd = os.getcwd()
filename = os.path.join(cwd, 'resonance_data', 'device_f_vs_B.csv')
BtoF = np.loadtxt(filename, delimiter = '\t',skiprows=1)
filename_exp = os.path.join(cwd, 'resonance_data', 'device_f_vs_B_exp.csv')
BtoF_exp = np.loadtxt(filename, delimiter = '\t',skiprows=1)

#### 1. Create instances of relevant instruments

In [2]:
#Magnet
#By = find_or_create_instrument()

#RF Source for reflectometry
#RFSource = find_or_create_instrument()  

# Oscilloscope for reflectometry
#lcr = find_or_create_instrument() 

#### 2. Define functions

In [23]:
def updateBtoFexp(B,rf):
    '''
    Updates and saves experimentally found optimal drive frequency.
    '''
    filename_exp = os.path.join(cwd, 'resonance_data', 'device_f_vs_B_exp.csv')
    BtoF_exp = np.loadtxt(filename, delimiter = '\t',skiprows=1)
    
    BmatchFound = False
    for idx, BandFpair in enumerate(BtoF_exp):
        if np.allclose(BandFpair[0],B): # update F for relevant B
            BtoF_exp[idx][1] = rf
            BmatchFound = True
    
    if not BmatchFound: # append new B,F pair
        BtoF_exp = np.concatenate((BtoF_exp,np.array([[B,rf]])))
    
    pd.DataFrame(BtoF_exp, columns= ['B (T)','freq (Hz)']).to_csv(filename_exp, index=False, sep='\t')
    return

In [24]:
def SignalOpt(aveSigOpt = 50, waitTime = 2):
    '''
    SignalOpt optimises a reflectometry signal for maximum phase response by varying the drive frequency. 
    
    IMPORTANT: 
        A look-up array with B field values in 1st column and corresponding approximate drive frequencies in 2nd column 
        must be defined under the name BtoF before running the function. 
    
    Args:
        aveSigOpt: Number of oscilloscope trace averages used when measuring the signal being optimised
        waitTime:  Number of seconds between setting new drive frequency and oscilloscope trace measurement
    '''
    #Check if the magnetic field is ramping. If it is, wait, if not, get B value 
    #which is used to look up the appropriate drive frequency
    while True:
        if By.ramping_state() == 'holding':
            B = By.field()
            B = np.abs(round(B*1000)/1000)
            break
        else:
            time.sleep(1)
    print("Magnetic Field = %s T" % (round(B*1000)/1000))
    
    #Initialise the signal optimisation variable and save current avering
    maxx = 0
    averaging = lcr.reflectometry.average()
    
    i = np.array(np.nonzero(BtoF[:,0] > B)).flat[0]    #Go trough look-up of B and drive frequency values
    if abs(BtoF[i,0] - B) < abs(BtoF[i-1,0] - B):
        closest = i                                  #Select the entry with closest match to B field
    else:
        closest = i-1
    rf = BtoF[closest, 1]    #Choose the matching drive frequency, rf
    
    #for entry in BtoF:                                      #Go trough look-up of B and drive frequency values
    #    if (abs(entry[0]-B)) <= 0.011:                        #Select the entry with closest match to B field
    #        rf = entry[1]                                   #Choose the matching drive frequency, rf
            
    #Step through a frequency range of rf +/- 5MHz, save maximum phase response and reassign drive frequency
    for f in np.arange(rf-5e6,rf+5e6,5e5):          
        RFSource.freq(f/1000000)
        time.sleep(waitTime)
        lcr.reflectometry.average(aveSigOpt)
        data = lcr.reflectometry.curvedata.get()
        #tempMax = np.amax(abs(np.subtract(data[3], 45)))
        tempMax = np.amax(np.subtract(data[3], 45))
        if tempMax > maxx:
            #Print when a new signal maximum is found and indicate the accompanying drive frequency
            print("New signal max of %s degree shift found at %s MHz" % (round(tempMax*1000)/1000,f/1e6))
            maxx = tempMax                           #Redefine max phase response
            rf = f                                   #Redefine rf to the rough signal optimisation value

    #Step through a frequency range of newly optimised rf +/- 0.5MHz in finer steps and reassign drive frequency
    for f in np.arange(rf-4e5,rf+5e5,1e5):
        RFSource.freq(f/1000000)
        time.sleep(waitTime)                
        lcr.reflectometry.average(aveSigOpt)
        data = lcr.reflectometry.curvedata.get()
        #tempMax = np.amax(abs(np.subtract(data[3], 45)))
        tempMax = np.amax(np.subtract(data[3], 45))
        if tempMax > maxx:
            #Print when a new signal maximum is found and indicate the accompanying drive frequency
            print("New signal max of %s degree shift found at %s MHz" % (round(tempMax*1000)/1000,f/1e6))
            maxx = tempMax                           #Redefine max phase response
            rf = f                                   #Redefine rf to the rough signal optimisation value
    
    #Signal now optimised. Optimised drive frequency printed and sent to RF source.
    print("The optimal drive frequency at %s T is %s MHz" % (B,rf/1e6))                
    RFSource.freq(rf/1000000)
    lcr.reflectometry.average(averaging)
    
    # Update and save experimentally optimal drive frequency
    updateBtoFexp(B,rf)
    
    return

In [ ]:
def FastSignalOpt():
    '''
    Optimises reflectometry signal by look-up into previously optimised f vs B only.
    '''
    while True:
        if By.ramping_state() == 'holding':
            B = By.field()
            B = np.abs(round(B*1000)/1000)
            break
        else:
            time.sleep(1)
    
    i = np.array(np.nonzero(BtoF_exp[:,0] > B)).flat[0]    #Go trough look-up of B and drive frequency values
    if abs(BtoF_exp[i,0] - B) < abs(BtoF_exp[i-1,0] - B):
        closest = i                                  #Select the entry with closest match to B field
    else:
        closest = i-1
    rf = BtoF_exp[closest, 1]  
    
    print(round(rf)/1000000)
    RFSource.freq(round(rf)/1000000)
    return

#### 3. Use functions
Run the function before a measurement

In [ ]:
SignalOpt()
dataid, ax, cbax = do0d(exp, lcr.reflectometry.curvedata, name = 'name')

or before each inner loop if e.g. doing a 2D measurement

In [ ]:
dataid, ax, cbax = do1d(exp, By.field, 0, 0.9, 46, 0.01, lcr.reflectometry.curvedata, before_inner_actions = (SignalOpt, ), name = 'name')

# Functions for finding and storing interdot charge transition (ICT) centre

In [9]:
from scipy.signal import argrelextrema
import numpy as np
from lmfit import Model
from matplotlib import pyplot as plt
from scipy import signal

# Load or make ICT dictionary for storing coordinates and dimensions of ICT
from functions.ICTDictionary import Dictionary
ICTDict = Dictionary('device_name')

We found no file with the name device_name.csv! An empty dictionary has been prepared in stead.


We can easily add an ICT to the dictionary as a test using format for the voltages (G1_centre, G1_length, G2_centre, G2_length):

In [10]:
ICTDict.addICT((2,0),(1,0.04,2,0.05))

ICT (2, 0) has been successfully added to the device_name dictionary with values (1, 0.04, 2, 0.05)


Below we define a bunch on functions that allow us to find the coordinates and size of an ICT

In [7]:
####################################### CUSTOM ERRORS #######################################
class VoltageRangeError(ValueError):
    """Exception raised for errors in the input."""
    
class BackgroundNoiseError(ValueError):
    """Exception raised for errors in the input."""
    
class SwitchError(RuntimeError):
    """Exception raised for errors in the input."""

####################################### MODELS #######################################
def gaussian(x, amp, cen, wid, offset):
    return amp * np.exp(-(x-cen)**2 / (2*wid**2)) + offset

def linear(x, a, b):
    return a*x + b

def mexicanhat(x, amp, cen, wid, offset):
    # for fitting phase data
    t = x - cen
    hat = 1 - ((t/wid)**2)
    return (hat*gaussian(x, amp, cen, wid, 0)) + offset

####################################### MODEL FITTERS #######################################
def hist_gaussian_fitter(z, z_type, nbins = 200, verbose = False):
    hist, binedges = np.histogram(z, bins = nbins)
    bincentres = binedges + (binedges[1]-binedges[0])/2
    bincentres = bincentres[:nbins]
    model = Model(gaussian)
    model.set_param_hint('offset', value=0, min=-1e-10, max=1e-10)        
    model.set_param_hint('amp', value=np.amax(hist))  
    model.set_param_hint('cen', value=bincentres[np.argmax(hist)])
    model.set_param_hint('wid', value=(bincentres[-1]-bincentres[0])/20)
    fit = model.fit(hist, x=bincentres)
    
    if verbose:
        print(fit.fit_report())
        plt.figure()
        plt.plot(bincentres, hist)
        plt.plot(bincentres, fit.init_fit, 'k--', label='initial fit')
        plt.plot(bincentres, fit.best_fit, 'r-', label='best fit')
        z_legend = ['I (V)','Q (V)','Magnitude (V)','Phase (deg)']
        plt.xlabel(z_legend[z_type])
        plt.ylabel('Histogram frequency with {0} bins'.format(nbins))
        plt.legend()
    return fit

def linear_fitter(x, y, weights = None):
    model = Model(linear)
    a_est = (y[-1]-y[0])/(x[-1]-x[0])
    b_est = y[0] - a_est*x[0]
    model.set_param_hint('a', value=a_est)        
    model.set_param_hint('b', value=b_est)   
    fit = model.fit(y, x=x, weights=weights)
    return fit

def mexicanhatfitter(trace_x, trace_y):
    # fits mexican hat func. to 1D voltage sweep across a charge transition. Works well on phase data.
    # 'trace' is a 1-dimensional np.array 
    # returns lmfit Model result object
    
    gmodel = Model(mexicanhat)
    xrange = np.abs(trace_x[-1] - trace_x[0])
    
    baseline = np.mean(trace_y)
    peakIdx = np.argmax(np.abs(trace_y - baseline))
    cen_est = trace_x[peakIdx]                          # take most distant point from mean as estimate for peak centre
    halfamp = np.abs(trace_y[peakIdx] - baseline)/2
    
    # detect if peak is maximum or minimum - important for setting bounds on fit params
    if (trace_y[peakIdx] - baseline) > 0:
        # peak is a maximum
        # estimate width - essential as ICT can be much thinner than DTL
        halfmax = baseline + halfamp
        try:
            Redge = np.array(np.where((trace_y[peakIdx:-1] - halfmax) < 0))[0][0]
            Ledge = np.array(np.where(np.flip(trace_y[0:peakIdx], axis=0) - halfmax < 0))[0][0]  # flip() used to count away from peak
            wid_est = ((Redge + Ledge)/len(trace_x))*xrange
        except:
            # width estimation sometimes breaks, especially if peak too close to edge of trace. Measurements shouldn't stop if this happens
            wid_est = 0.1*xrange
        
        amp_est = trace_y[peakIdx] - np.amin(trace_y)    # estimate amplitude
        gmodel.set_param_hint('offset', value=baseline, min=-np.inf, max=baseline)
        
    else:
        # peak is a minimum
        halfmin = baseline - halfamp
        try:
            Redge = np.array(np.where((trace_y[peakIdx:-1] - halfmin) > 0))[0][0]
            Ledge = np.array(np.where(np.flip(trace_y[0:peakIdx], axis=0) - halfmin > 0))[0][0]
            wid_est = ((Redge + Ledge)/len(trace_x))*xrange
        except:
            wid_est = 0.1*xrange
        
        amp_est = trace_y[peakIdx] - np.amax(trace_y)
        gmodel.set_param_hint('offset', value=baseline, min=baseline, max=np.inf)
        
    gmodel.set_param_hint('amp', value=amp_est, min=0, max=amp_est)    # beware hard-coded max and min assumptions!
    gmodel.set_param_hint('cen', value=cen_est, min=cen_est-0.2*xrange, max=cen_est+0.2*xrange)
    gmodel.set_param_hint('wid', value=wid_est, min=0, max=0.5*xrange)
    
    result = gmodel.fit(trace_y, x=trace_x)
    return result

####################################### DATA IMPORT #######################################
def load_least_noisy_data(exp, run_id = None, stds = 3, verbose = False):
    '''
    Loads the least noisy data type (I, Q, Magnitude, or Phase) and returns the x, y, z and data type.
    
    params
    ------
    exp: qcodes experiment instance
    run_id: measurement id to load
    stds: threshold for determining least noisy data
    '''
    if run_id == None:
        run_id = exp.last_counter
    data = exp.data_set(run_id).get_parameter_data()
    
    ###################################################################################
    ## ----------------------------------------------------------------------------- ##
    ## IMPORTANT: LOADING DATA DEPENDS ON YOUR DATASTRUCTURE IN THE AQUIRED DATA !!! ##
    ## ----------------------------------------------------------------------------- ##
    ###################################################################################
    
    x = data['I']['G1_volt']
    y = data['I']['LCR_Reflectometry_Sweep']
    z_I = data['I']['I']
    z_Q = data['Q']['Q']
    z_M = data['magnitude']['magnitude']
    z_P = data['phase']['phase']
    z = [z_I, z_Q, z_M, z_P]
    z_legend = ['I','Q','Magnitude','Phase']
    
    try:
        x_thresholded_I,_,_ = threshold(x, y, z_I, 0, stds = stds, verbose = verbose)
    except:
        x_thresholded_I = np.array([])
    
    try:
        x_thresholded_Q,_,_ = threshold(x, y, z_Q, 1, stds = stds, verbose = verbose)
    except:
        x_thresholded_Q = np.array([])
        
    try:
        x_thresholded_M,_,_ = threshold(x, y, z_M, 2, stds = stds, verbose = verbose)
    except:
        x_thresholded_M = np.array([])
        
    try:
        x_thresholded_P,_,_ = threshold(x, y, z_P, 3, stds = stds, verbose = verbose)
    except:
        x_thresholded_P = np.array([])
    
    i = np.argmax([x_thresholded_I.shape[0],x_thresholded_Q.shape[0],x_thresholded_M.shape[0],x_thresholded_P.shape[0]])
    
    print('The least noisy data (' + z_legend[i] + ') has been successfully loaded')
    return x, y, z[i], i

####################################### FIND ICT FUNCTIONS #######################################
def get_ict(x, y, z, z_type, stds, verbose = False):
    '''
    Finds and returns the coordinates of the ICT in the x, y, z data. 
    
    params
    ------
    x: array with x values (gate 1)
    y: array with y values (gate 2)
    z: array of dimension x by y with reflectometry data
    z_type: int with mapping 0: I, 1: Q, 2: Mag, 3: Phase
    stds: float indicating where to threshold the data (large value = less data is removed). stds=6 may be a good starting point.
    ''' 
    n_traces = x.shape[0]/6000
    
    threshold_trace_ratio = 0
    while threshold_trace_ratio < 0.80: # require that thresholding leaves 80% of the traces intact     
        try:
            # remove all data below threshold
            x_thresholded_nozeros, y_thresholded_nozeros, z_thresholded_norm_nozeros = threshold(x, y, z, z_type, stds = stds, verbose = verbose)

            # take the mean for each trace (one trace may have several point outside of the threshold)
            x_transition, y_transition = trace_mean(x_thresholded_nozeros, y_thresholded_nozeros, weights = z_thresholded_norm_nozeros)
            
            threshold_trace_ratio = x_transition.shape[0]/n_traces
            stds -= 0.5
        except ValueError:
            raise BackgroundNoiseError('When removing gaussian noise from the data, there is no data left after thresholding!')
        except SwitchError:
            raise SwitchError('A switch seems to have occured in the measurement!')
    
    switchbool = check_for_switch(x_transition, y_transition, z_type, stds = 8, verbose = verbose)
    if switchbool:
        raise SwitchError('A switch seems to have occured in the measurement!')        

    # get the index position of the triple points
    tp_args, x_transition, y_transition, x_outliers_removed, y_outliers_removed, z_norm_outliers_removed = get_triple_point_args(x_thresholded_nozeros, y_thresholded_nozeros, weights = z_thresholded_norm_nozeros, verbose = verbose)
    
    # determine ict dimensions
    tp_1 = (x_transition[tp_args[0]], y_transition[tp_args[0]])
    tp_2 = (x_transition[tp_args[1]], y_transition[tp_args[1]])
    ict_centre = ((tp_1[0]+tp_2[0])/2,(tp_1[1]+tp_2[1])/2)
    ict_y_length = tp_2[1]-tp_1[1]
    ict_x_length = tp_2[0]-tp_1[0]
    
    # prepare colour and transparency for threshold data
    rgba_colors_outliers_removed = np.zeros((z_norm_outliers_removed.shape[0],4))
    rgba_colors_outliers_removed[:, 2] = 1.0
    rgba_colors_outliers_removed[:, 3] = z_norm_outliers_removed
    
    # plot transition, triple points and ict centre
    plt.figure()
    plt.scatter(x_outliers_removed, y_outliers_removed, s = 2, color=rgba_colors_outliers_removed, label = 'Thresholded data')
    plt.plot(x_transition, y_transition, 'tab:orange', label = ' Transition mean')
    plt.xlim([np.amin(x_outliers_removed),np.amax(x_outliers_removed)])
    plt.ylim([np.amin(y_outliers_removed),np.amax(y_outliers_removed)])
    plt.scatter([tp_1[0],tp_2[0]], [tp_1[1],tp_2[1]], c = 'r', label = 'Estimated triple points')
    plt.scatter(ict_centre[0], ict_centre[1], c = 'g', label = 'Estimated ICT centre')
    plt.xlabel('Step voltage (V)')
    plt.ylabel('Sweep voltage (V)')
    plt.legend()
    plt.tight_layout()
    
    ###################### Get name and location for figure ######################
    cwd = os.getcwd()
    name = os.path.join(cwd, exp.name, exp.sample_name, 'png', str(exp.last_data_set().run_id)+'_findICTcentre.png')
    plt.savefig(name)
    plt.show()
    
    # (G1_centre, G1_length, G2_centre, G2_length)
    return (round(ict_centre[0]*1e8)/1e8, round(ict_x_length*1e8)/1e8, round(ict_centre[1]*1e8)/1e8, round(ict_y_length*1e8)/1e8)

def threshold(x, y, z, z_type, stds = 6, verbose = False):
    '''
    Thresholds 2D data
    '''
    gauss_fit = hist_gaussian_fitter(z, z_type, nbins = 200, verbose = verbose)
    lower_threshold = gauss_fit.best_values['cen']-gauss_fit.best_values['wid']*stds
    upper_threshold = gauss_fit.best_values['cen']+gauss_fit.best_values['wid']*stds
    z_thresholded = np.where(z > upper_threshold, z, 0) + np.where(z < lower_threshold, z, 0)
    z_thresholded_norm = normalise(z_thresholded)
    
    x_thresholded = np.where(z_thresholded > 0, x, 0)
    y_thresholded = np.where(z_thresholded > 0, y, 0)
    
    z_thresholded_norm_nozeros = z_thresholded_norm[np.where(z_thresholded > 0)]
    x_thresholded_nozeros = x_thresholded[np.where(x_thresholded > 0)]
    y_thresholded_nozeros = y_thresholded[np.where(y_thresholded > 0)]
    
    if verbose:
        plt.plot([gauss_fit.best_values['cen']+gauss_fit.best_values['wid']*stds, gauss_fit.best_values['cen']+gauss_fit.best_values['wid']*stds],[0,gauss_fit.best_values['amp']], 'tab:orange', label = 'threshold')
        plt.legend()
        
    return x_thresholded_nozeros, y_thresholded_nozeros, z_thresholded_norm_nozeros

def threshold_1d(x, y, y_type, stds = 6, verbose = False):
    '''
    Thresholds 1D data
    '''
    try:
        gauss_fit = hist_gaussian_fitter(y, y_type, nbins = 200, verbose = verbose)
        lower_threshold = gauss_fit.best_values['cen']-gauss_fit.best_values['wid']*stds
        upper_threshold = gauss_fit.best_values['cen']+gauss_fit.best_values['wid']*stds
        y_thresholded = np.where(y > upper_threshold, y, 0) + np.where(y < lower_threshold, y, 0)
        y_thresholded_norm = normalise(y_thresholded)

        x_thresholded = np.where(y_thresholded > 0, x, 0)

        y_thresholded_norm_nozeros = y_thresholded_norm[np.where(y_thresholded > 0)]
        x_thresholded_nozeros = x_thresholded[np.where(x_thresholded > 0)]

        if verbose:
            plt.plot([gauss_fit.best_values['cen']+gauss_fit.best_values['wid']*stds, gauss_fit.best_values['cen']+gauss_fit.best_values['wid']*stds],[0,gauss_fit.best_values['amp']], 'tab:orange', label = 'threshold')
            plt.legend()

        return x_thresholded_nozeros, y_thresholded_norm_nozeros
    except:
        return np.array([]), np.array([])

def get_triple_point_args(x, y, weights = None, verbose = False):
    '''
    Finds the triple points of the ICT
    '''
    success = False
    n_outlier_removals = 0
    while not success and n_outlier_removals < 20:
        try:
            x_transition, y_transition = trace_mean(x, y, weights = weights)
            lin_fit = linear_fitter(x_transition, y_transition)
            error_0 = linerror(y_transition, lin_fit.best_fit)

            if verbose:
                plt.figure()
                plt.plot(x_transition, y_transition, '.')
                plt.plot(x_transition, lin_fit.best_fit)
                plt.plot(x_transition, y_transition, '.', label = 'Transition')
                plt.plot(x_transition, lin_fit.init_fit, 'k--', label='initial fit')
                plt.plot(x_transition, lin_fit.best_fit, 'r-', label='best fit')
                plt.xlabel('Step voltage (V)')
                plt.ylabel('Sweep voltage (V)')
                plt.legend()
                plt.tight_layout()
                plt.show()                

            error = error_0
            num_extremas = 0
            n_smooths = 0
            while num_extremas != 2:
                n_smooths += 1
                if n_smooths > 8: # warning: n_smooths below three may cause low-SNR icts not to be identified
                    raise VoltageRangeError('Voltage window in stepped voltage axis is too small or shifted too far from ICT centre')
                window = np.max((len(lin_fit.best_fit)//40*2-1,3))
                error = signal.savgol_filter(error, window, 1)
                max_args = argrelextrema(error, np.greater)[0]
                min_args = argrelextrema(error, np.less)[0]
                num_extremas = max_args.shape[0] + min_args.shape[0]

            if verbose:        
                plt.figure()
                plt.plot(x_transition, error_0, label = 'Unsmoothed error')
                plt.plot(x_transition, error, label = 'Error smoothed {0} time(s)'.format(n_smooths))
                plt.xlabel('Step voltage (V)')
                plt.ylabel('Error from linear fit (V)')
                plt.legend()
                plt.tight_layout()
                plt.show()
                
            success = True
            
        except:
            x, y, weights = remove_outliers_lin_fit(x, y, weights = weights, percent = 2, verbose = verbose)
            n_outlier_removals += 1
            if x.shape[0] < 50:
                raise BackgroundNoiseError('When removing outliers from the data, there is not enough data left')

            if verbose:        
                plt.figure()
                plt.plot(x_transition, error_0, label = 'Unsmoothed error')
                plt.plot(x_transition, error, label = 'Error smoothed {0} time(s)'.format(n_smooths))
                plt.xlabel('Step voltage (V)')
                plt.ylabel('Error from linear fit (V)')
                plt.legend()
                plt.tight_layout()
                plt.show()
    
    return np.concatenate((min_args, max_args)), x_transition, y_transition, x, y, weights

def normalise(z):
    non_zero_min = np.min(z[np.where(z > 0)])
    z = np.where(z > 0, z - non_zero_min, z)
    return z / np.amax(np.abs(z))

def sqerror(t, t_model):
    return (t-t_model)**2

def linerror(t, t_model):
    return (t-t_model)

def trace_mean(x, y, weights = None):
    prev_x = 0
    x_no_replicas = []
    y_mean = []
    for curr_x in x:
        if prev_x < curr_x:
            if not (weights is None):
                y_mean.append(np.average(y[np.where(x == curr_x)], weights = weights[np.where(x == curr_x)]))
            else:
                y_mean.append(np.mean(y[np.where(x == curr_x)]))
            x_no_replicas.append(curr_x)
            prev_x = curr_x

    x_no_replicas = np.array(x_no_replicas)
    y_mean = np.array(y_mean)
    return x_no_replicas, y_mean

def remove_outliers_lin_fit(x, y, weights = None, percent = 5, verbose = False):
    '''
    Removes a percentage of the data that is farthest from a linear fit to the data.
    
    params
    ------
    x: array with the x indexes of a transition
    y: array with the y indexes of a transition
    weights: array with the weighting of each data point
    percent: float indicating how much of the data to remove
    '''
    lin_fit = linear_fitter(x, y, weights = weights)
    error = sqerror(y, lin_fit.best_fit)
    
    sort_order = np.argsort(error)
    percent_cut = np.int(np.round((1-percent/100)*x.shape[0]))
    x_outliers_removed = x[sort_order][:percent_cut]
    y_outliers_removed = y[sort_order][:percent_cut]
    weigths_outliers_removed = weights[sort_order][:percent_cut]
    
    rev_sort_order = np.argsort(x_outliers_removed)
    x_outliers_removed = x_outliers_removed[rev_sort_order]
    y_outliers_removed = y_outliers_removed[rev_sort_order]
    weigths_outliers_removed = weigths_outliers_removed[rev_sort_order]
    
    if verbose:
        plt.figure()
        plt.plot(x, y, 'o', label = 'Before outlier removal')
        plt.plot(x, lin_fit.best_fit, label = 'Linear fit for error calculation')
        plt.plot(x_outliers_removed, y_outliers_removed, '.', label = 'After outlier removal')
        plt.xlabel('Step voltage (V)')
        plt.ylabel('Sweep voltage (V)')
        plt.legend()
        plt.tight_layout()
    
    return x_outliers_removed, y_outliers_removed, weigths_outliers_removed


def check_for_switch(x, y, y_type, stds = 8, verbose = False):
    '''
    Checks if there was a switch during the measurement of the ICT, as a switch may be mistaken as an ICT.
    
    params
    ------
    x: array with x data for transition
    y: array with y data for transition
    y_type: int with mapping 0: I, 1: Q, 2: Mag, 3: Phase
    stds: float indicating where to threshold the data (large value = less data is removed).
    '''
    
    y_dif = np.ones(y.shape[0]-1)
    y_difdif = np.ones(y.shape[0]-2)

    for i in range(y.shape[0]-1):
        y_dif[i] = (y[i+1] - y[i])/(x[i+1] - x[i])

    for i in range(y_dif.shape[0]-1):
        y_difdif[i] = (y_dif[i+1] - y_dif[i])/(x[i+1] - x[i])

    if verbose:
        fig, ax1 = plt.subplots()
        ax1.set_xlabel('Step Voltage (V)')
        ax1.set_ylabel('Slope of transition', color='tab:red')
        ax1.plot(x[:-1], y_dif, '.', color='tab:red')
        ax1.tick_params(axis='y', labelcolor='tab:red')

        ax2 = ax1.twinx()  # create a second axes that shares the same x-axis
        color = 'tab:blue'
        ax2.set_ylabel('Curvature of transition', color='tab:blue')  # we already handled the x-label with ax1
        ax2.plot(x[:-2], y_difdif, '.', color='tab:blue')
        ax2.tick_params(axis='y', labelcolor='tab:blue')

        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()
        
        #plt.figure()
        #plt.plot(x[:-1], y_dif, '.', label = 'Slope of transition')
        ##plt.plot(x[:-2], y_difdif, '.', label = 'Curvature of transition')
        #plt.xlabel('Step Voltage (V)')
       # plt.legend()

    x_switch, y_switch = threshold_1d(x[:-2], y_difdif, y_type, stds = stds, verbose = verbose)
    x_switch2, y_switch2 = threshold_1d(x[:-2], -y_difdif, y_type, stds = stds, verbose = verbose)

    if verbose:
        print('Number of points with abnormally large transition curvature: {0}'.format(x_switch.shape[0], x_switch2.shape[0]))

    if (x_switch.shape[0] == x_switch2.shape[0] and x_switch.shape[0] == 1):
        return True
    else:
        return False  

If we already have a measurement of an ICT, the key functions we need to find and save the coordinates and size are now just:

In [ ]:
ict = (2,1)
last_id = exp.last_counter
x, y, z, z_type = load_least_noisy_data(exp, run_id = last_id)
centre = get_ict(x, y, z, z_type, stds = 6, verbose = verbose)
ICTDict.addICT(ict, centre)

However, it is also useful to be able run just a simple function that both acquires a charge stability diagram measurement and finds the ICT. This is done below with the function findICTwrapped:

In [ ]:
def finetune_sweep_centre(ict_centre, data_index, verbose = False):
    '''
    Finetunes the centre position of the ICT on the sweep axis (corresponing the the gate with resonant circuit).
    
    params
    ------
    ict_centre : tuple (G1_centre, G1_length, G2_centre, G2_length)
    data_index : int with mapping 0: I, 1: Q, 2: Mag, 3: Phase
    '''
    pre_ave = lcr.reflectometry.average()
    pre_power = RFSource.power()
    RFSource.power(10)
    lcr.reflectometry.average(200)
    
    G2.volt(ICTDict.dict[ict]['G2_centre'])
    G2.amp(5*ICTDict.dict[ict]['G2_length'])
    G1.volt(ict_centre[0])
    y = lcr.reflectometry.curvedata.get()    # get data
    x = np.array(lcr.reflectometry.curvedata.calc_set_points()[0])    # get x-axis
    
    x_threshold, y_threshold = threshold_1d(x, y[data_index], data_index, stds = 3, verbose = verbose)
    threshold_ratio = x_threshold.shape[0]/y[data_index].shape[0]
    print('Ratio of thresholded data to total data in finetuning of fit: %.6f' % threshold_ratio)    
    if threshold_ratio > 0.0002:
        fit = mexicanhatfitter(x, y[data_index])
        if verbose:
            print(fit.fit_report())
            plt.figure()
            plt.plot(x, y[data_index])
            plt.plot(x, fit.init_fit, 'k--', label='initial fit')
            plt.plot(x, fit.best_fit, 'r-', label='best fit')
            plt.legend()
        
        lcr.reflectometry.average(pre_ave)
        RFSource.power(pre_power)
        return (ict_centre[0], ict_centre[1], round(fit.best_values['cen']*1e8)/1e8, ict_centre[3])
    
    else:
        lcr.reflectometry.average(pre_ave)
        RFSource.power(pre_power)
        return (ict_centre[0], ict_centre[1], ict_centre[2], ict_centre[3])
    
def findICTwrapped(ict, exp, verbose = False):
    pre_ave = lcr.reflectometry.average()
    pre_power = RFSource.power()
    RFSource.power(10)
    
    setSweepGate(offset = ICTDict.dict[ict]['G2_centre']*4, amp = 2.5)
    _ave_start = 25
    _G1centre = ICTDict.dict[ict]['G1_centre']
    _G1offset = 0.0075 ## HYPERPARAMETER FOR CHANGING WIDTH OF STEP GATE WINDOW IN SEARCH OF ICT
    _n_traces = np.int(_G1offset*10000+1)
    r = 1
    s = 0
    ICTfound = False
    while not (r > 3 or s > 5 or ICTfound):
        try:
            # set voltages for stability diagram measurement
            G1start = _G1centre-_G1offset*r
            G1end = _G1centre+_G1offset*r
            lcr.reflectometry.average(r*25)
            # do charge stability diagram measurement
            dataid, ax, cbax = do1d(exp, G1.volt, G1start, G1end, _n_traces*r, 0.01, lcr.reflectometry.curvedata, name = name)
            plt.show()
            
            # update centre of ICT
            last_id = exp.last_counter
            x, y, z, z_type = load_least_noisy_data(exp, run_id = last_id)
            centre = get_ict(x, y, z, z_type, stds = 6, verbose = verbose)
            ICTfound = True
            
        except SwitchError:
            print('A switch occured and was identified. Repeating measurement.')
            s += 1
            
        except:
            print('Measurement was too noisy or ICT was not inside measurement window. Expand measurement window and increase averaging for next measurement.')
            r += 1
            
#       TO-DO
#         finally:
#             if not ICTfound:
#                 send slack message
    
    RFSource.power(pre_power)
    lcr.reflectometry.average(pre_ave)
    
    return centre, z_type

To run a measurement and add ICT to dictionary, we just run

In [ ]:
centre, _ = findICTwrapped(ict, exp, verbose = False)
ICTDict.addICT(ict,centre_fine)

Or if we want to update one or more ICT positions, we just run

In [ ]:
# update centre of ICT
for ict in [(2,1)]:
    centre, i = findICTwrapped(ict, exp, verbose = False)
    centre_fine = finetune_sweep_centre(centre, i, verbose = False)
    ICTDict.addICT(ict,centre_fine)

An updated ICT dictionary is incredibly useful, as you can loop through a subset of the ICTs in your dictionary and perform the same measurement on each ICT using the stored information in the dictionary. Example below

In [ ]:
# voltage source for gate 1
#G1 = find_or_create_instrument() 

# function generator for gate 2
#G2 = find_or_create_instrument() 

In [ ]:
for ict in [(1,1),(2,1),(1,2),(2,2)]:
    # set voltages based on information about ICT coordinates
    G1.volt(ICTDict.dict[ict]['G1_centre'])
    G2.volt(ICTDict.dict[ict]['G2_centre'])
    G2.amp(5*ICTDict.dict[ict]['G2_length'])
    
    # do measurement
    dataid, ax, cbax = do1d(exp, By.field, 0, 0.9, 46, 0.01, lcr.reflectometry.curvedata, before_inner_actions = (SignalOpt, ), name = 'name')plt.show()
    
    # reset
    By.field(0)